In [ ]:
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import HistGradientBoostingClassifier, AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.metrics import brier_score_loss
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from imblearn.metrics import classification_report_imbalanced
#from imblearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost import XGBClassifier
from imblearn.under_sampling import NearMiss

### Load binary unbalanced data

In [ ]:
def binaryBalancedData():
    file = 'Datasets/diabetes_binary_health_indicators_BRFSS2015.csv'
    df = pd.read_csv(file)
    
    #Drop duplicates
    df.drop_duplicates(inplace=True)
    
    #Change all column names to lower case
    df.columns = df.columns.str.lower().str.replace('diabetes_binary','diabetes')

    return df

def splitDF(df):    
    target = df.diabetes
    data = df.drop(columns=['diabetes'])
    dfTrainFull, dfTest, yTrainFull, yTest = train_test_split(data, target, test_size=0.2, random_state=11)
    dfTrain, dfVal, yTrain, yVal = train_test_split(data, target, test_size=0.25, random_state=11)
    return dfTrainFull, yTrainFull, dfTrain, yTrain, dfVal, yVal, dfTest, yTest


In [ ]:
df = binaryBalancedData()
#dfTrainFull, yTrainFull, dfTrain, yTrain, dfVal,yVal, dfTest, yTest = splitDF(df)
df.head()

In [ ]:
# newdf = df.drop(columns=['nodocbccost', 'fruits', 'anyhealthcare', 'sex', 'veggies', 'smoker', 'menthlth','physhlth'])

In [ ]:
dfTrainFull, yTrainFull, dfTrain, yTrain, dfVal, yVal, dfTest, yTest = splitDF(newdf)

### Preprocessing

In [ ]:
categoricalCols = ['highbp', 'highchol', 'cholcheck','smoker',
       'stroke', 'heartdiseaseorattack', 'physactivity', 'fruits', 'veggies',
       'hvyalcoholconsump', 'anyhealthcare', 'nodocbccost', 'genhlth','diffwalk', 'sex', 'education']

numericalCols = ['bmi', 'menthlth', 'physhlth', 'income']

In [ ]:
#dfTrain.income = np.log1p(dfTrain.income)

In [ ]:
# creating preprocesors
from sklearn.preprocessing import OneHotEncoder, StandardScaler
catPreprocessor = OneHotEncoder(handle_unknown="ignore")
numPreprocessor = StandardScaler()

# Transforming the data
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer([
    #('one-hot-encoder', catPreprocessor, categoricalCols)],remainder="passthrough")
    #('one-hot-encoder', catPreprocessor, categoricalCols),('standard_scaler', numPreprocessor, numericalCols)])
    ], remainder="passthrough")


### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model = KNeighborsClassifier(n_neighbors=3)
model.fit(dfTrain, yTrain)

In [ ]:
yValPredProba = model.predict_proba(dfVal)[:,1]
yValPred = model.predict(dfVal)
auc = roc_auc_score(yVal, yValPredProba)
print(f'Val AUC: {auc}')

yTestpred=model.predict_proba(dfTest)[:,1]
print(f'Test AUC: {roc_auc_score(yTest, yTestpred)}')

yTrainPredProba = model.predict_proba(dfTrain)[:,1]
yTrainPred = model.predict(dfTrain)
print(f'Train AUC: {auc}')

print(classification_report(yVal, model.predict(dfVal), ))
print(classification_report_imbalanced(yVal, model.predict(dfVal)))

In [ ]:
for p in model.get_params():
    print(p)

In [ ]:
model = Pipeline([("classifier", KNeighborsClassifier())])
param_grid = {
    'classifier__n_neighbors': (7,10,12,15) }
scoring = {"AUC": "roc_auc"}#, "f1_weighted": "f1_weighted"}

cv = StratifiedKFold(n_splits=3)
model_grid_search = GridSearchCV(model,
                                 param_grid=param_grid,
                                 #scoring=scoring,
                                 #scoring="f1_weighted",
                                 scoring="roc_auc",
                                 n_jobs=2,
                                 cv=cv,
                                 return_train_score=True,)
                                 #refit=False)
model_grid_search.fit(dfTrain, yTrain)

In [ ]:
print(model_grid_search.cv_results_.keys())

cv_results = pd.DataFrame(model_grid_search.cv_results_)
#cv_results
res = cv_results[[
    "param_classifier__n_neighbors",
    "mean_test_score",
    'std_test_score',
    'rank_test_score'
    #"mean_train_AUC", 
    #"std_train_AUC",
    #"rank_test_AUC",
    #"mean_train_f1_weighted",
    #"std_train_f1_weighted",
    #"rank_test_f1_weighted"
]]

res.sort_values(by=['rank_test_score']).head(10)

In [ ]:
display = PrecisionRecallDisplay.from_estimator(
    model_grid_search, dfVal, yVal, name="Validation", 
)
_ = display.ax_.set_title("2-class Precision-Recall curve")

In [ ]:
model = Pipeline([("classifier", KNeighborsClassifier())])
param_grid = {
    'classifier__n_neighbors': (3, 5,7,10,12) }
scoring = {"AUC": "roc_auc", "f1_weighted": "f1_weighted"}

cv = StratifiedKFold(n_splits=3)
model_grid_search = GridSearchCV(model,
                                 param_grid=param_grid,
                                 scoring=scoring,
                                 #scoring="f1_weighted",
                                 #scoring="roc_auc",
                                 n_jobs=2,
                                 cv=cv,
                                 return_train_score=True,
                                 refit=False)
model_grid_search.fit(dfTrainFull, yTrainFull)

In [ ]:
print(model_grid_search.cv_results_.keys())

cv_results = pd.DataFrame(model_grid_search.cv_results_)
#cv_results
res = cv_results[[
    "param_classifier__n_neighbors",
    #"mean_test_score",
    #'std_test_score',
    #'rank_test_score'
    "mean_train_AUC", 
    "std_train_AUC",
    "rank_test_AUC",
    "mean_train_f1_weighted",
    "std_train_f1_weighted",
    "rank_test_f1_weighted"
]]

res.sort_values(by=['rank_test_AUC','rank_test_f1_weighted']).head(10)

In [ ]:
model = Pipeline([("classifier", KNeighborsClassifier(n_neighbors=7))])
scoring = {"AUC": "roc_auc", "f1_weighted": "f1_weighted"}
cv = StratifiedKFold(n_splits=3)
#model_grid_search = cr(model,
#                                  param_grid=param_grid,
#                                 scoring=scoring,
#                                 #scoring="f1_weighted",
#                                 #scoring="roc_auc",
#                                 n_jobs=2,
#                                 cv=cv,
#                                 return_train_score=True,
#                                 refit=False)
#model_grid_search.fit(dfTrainFull, yTrainFull)


cv = StratifiedKFold(n_splits=3)
cv_results = cross_validate(model, dfTrainFull, yTrainFull,cv=cv,
                            scoring=['roc_auc', 'average_precision',
                                     'recall','f1_weighted'])
cv_results = pd.DataFrame(cv_results)
cv_results

In [ ]:
scoresMean = cv_results.mean(axis=0)     
scoresStd = cv_results.std(axis=0)    
for k, i,j in list(zip(cv_results.columns.values,scoresMean, scoresStd)):
    print(f'{k} - {i} +/- {j}')

In [ ]:


fig, ax = plt.subplots(figsize=(6, 4))
fpr, tpr, _ = roc_curve(yVal.values, yValPredProba)
roc_display1 = RocCurveDisplay(fpr=fpr, tpr=tpr).plot(ax=ax)
#fpr2, tpr2, _ = roc_curve(yVal, targetRand)
#roc_display2 = RocCurveDisplay(fpr=fpr2, tpr=tpr2).plot(ax=ax)

plt.show()

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeClassifier(random_state=0)
cv_results = cross_validate(tree, dfTrainFull, yTrainFull, n_jobs=2)
scores = cv_results["test_score"]

print(f"R2 score obtained by cross-validation: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")

In [ ]:
from sklearn.ensemble import BaggingRegressor

base_estimator = DecisionTreeClassifier(random_state=0, max_depth=7, max_features=9, max_leaf_nodes=20)
bagging_regressor = BaggingRegressor(base_estimator=base_estimator, n_estimators=20, random_state=0)

cv_results = cross_validate(bagging_regressor, dfTrain, yTrain, n_jobs=-1)
scores = cv_results["test_score"]

print(f"R2 score obtained by cross-validation: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

param_grid = {
    "max_depth": [5, 8, None],
    "min_samples_split": [2, 10, 30, 50],
    "min_samples_leaf": [0.01, 0.05, 0.1, 1]}

#cv = StratifiedKFold(n_splits=3)
cv = 3

tree = GridSearchCV(DecisionTreeClassifier(random_state=0),
                    param_grid=param_grid, cv=cv, n_jobs=2)
cv_results = cross_validate(tree, dfTrainFull, yTrainFull, n_jobs=-1,
                            return_estimator=True)
scores = cv_results["test_score"]

print(f"R2 score obtained by cross-validation: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")